In [1]:
import pandas as pd
import numpy as np

In [3]:
import keras

**Prepare Data**

Sequences will be transform as follow:


Let sequence be: $[x_1, x_2, x_3, x_4, x_5]$

where $x_i \in \{-1, 1\}$ where -1 corresponds to defection and 1 to a cooperation. 

In [4]:
def batch_generator(input_path, output_path, bs=2470):
    
    while True:
        skip = []
        for iterations in range(0, 202):
            if iterations > 0:
                skip += [x for x in range((iterations - 1) * bs, bs * iterations)]
            batch = pd.read_csv(input_path,
                                nrows=bs,
                                skiprows=skip,
                                index_col=0).values

            output_batch = pd.read_csv(output_path,
                                       nrows=bs,
                                       skiprows=skip,
                                       index_col=0).values        

            batch = np.array([[x for x in mini_batch if np.isnan(x) == False] for mini_batch in batch])
            output_batch = np.array([[x for x in mini_batch if np.isnan(x) == False] for mini_batch in output_batch])

            try:
                batch = batch.reshape((batch.shape[0], batch.shape[1], 1))
                output_batch = output_batch.reshape((output_batch.shape[0], output_batch.shape[1], 1))
            except IndexError:
                batch = batch.reshape((batch.shape[0], 1, 1))
                output_batch = output_batch.reshape((output_batch.shape[0], 1, 1))
        
            yield (batch, output_batch)

In [12]:
df = pd.read_csv('inputs_test.csv', index_col=0)

In [13]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,201
246031,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
246032,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
246033,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
246034,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
246035,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [6]:
trainGen = batch_generator('inputs_train.csv', 'outputs_train.csv', bs=10)
testGen =  batch_generator('inputs_test.csv', 'outputs_test.csv', bs=1218)

**Creating the model**

In [7]:
import tensorflow as tf
import numpy as np

In [8]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, CuDNNLSTM

In [9]:
model = Sequential()

model.add(CuDNNLSTM(1, return_sequences=True, input_shape=(None, 1)))

model.add(Dropout(rate=0.2))

model.add(Dense(1, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_2 (CuDNNLSTM)     (None, None, 1)           16        
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 1)           0         
_________________________________________________________________
dense_2 (Dense)              (None, None, 1)           2         
Total params: 18
Trainable params: 18
Non-trainable params: 0
_________________________________________________________________


In [17]:
trainGen = batch_generator('inputs_train.csv', 'outputs_train.csv')
testGen =  batch_generator('inputs_test.csv', 'outputs_test.csv', bs=1218)

history = model.fit_generator(trainGen,
                              steps_per_epoch=202,
                              epochs=3,
                              verbose=1,
#                               validation_data=testGen,
#                               validation_steps=202,
                              )

In [ ]:
model.save('/home/nightwing/Desktop/my_model.h5') 
model.save_weights('/home/nightwing/Desktop/my_model_weights.h5')

In [ ]:
model.predict(np.array([0, 1, 1]).reshape(1, 3, 1))

In [ ]:
model.predict(np.array([0, 1, 1, 1, 0, 0, 0]).reshape(1, 7, 1))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots();

plt.plot(history.history['acc'], label='accuracy', color='red', linestyle='--');
plt.plot(history.history['val_acc'], label=' validation accuracy');

plt.legend();

plt.savefig('accuracy_plot.pdf')

In [ ]:
fig, ax = plt.subplots();

plt.plot(history.history['loss'], label='loss', color='red', linestyle='--');
plt.plot(history.history['val_loss'], label=' validation loss');

plt.legend();

In [ ]:
from keras.models import load_model
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
model = load_model('/home/nikoleta/Desktop/my_model.h5')

In [ ]:
fig, ax = plt.subplots();

seq = [1 for _ in range(100)]
len_of_seq = len(seq)

probabilities = model.predict(np.array(seq).reshape(1, len_of_seq, 1)).reshape(len_of_seq)

plt.plot(abs(probabilities - 0.5));

In [ ]:
fig, ax = plt.subplots();

seq = [0 for i in range(100)]
len_of_seq = len(seq)

probabilities = model.predict(np.array(seq).reshape(1, len_of_seq, 1)).reshape(len_of_seq)

plt.plot(probabilities - 0.5);

**Unknown lenght**

In [12]:
import pandas as pd
import numpy as np

In [17]:
from sklearn.model_selection import train_test_split

In [45]:
def batch_generator(inputs, outputs, num_of_steps):
    while True:
        for size in range(1, num_of_steps):
            batches = [
                (sequence, target)
                for sequence, target in zip(inputs, outputs)
                if len(sequence) == size
            ]

            x, y = zip(*batches)
            batch = np.array(x)
            output_batch = np.array(y)

            try:
                batch = batch.reshape((batch.shape[0], batch.shape[1], 1))
                output_batch = output_batch.reshape((output_batch.shape[0], 1))
            except IndexError:
                batch = batch.reshape((batch.shape[0], 1, 1))
                output_batch = output_batch.reshape((output_batch.shape[0], 1))

            yield batch, output_batch

In [19]:
def format_sequences_to_input(sequences):
    inputs = sequences.drop(columns=["opponent", "gene_204"]).values
    max_length = len(inputs[0])

    prep_X_train = []
    for histories in range(1, max_length + 1):
        for sequence in inputs:
            assert len(sequence) == max_length
            prep_X_train.append(sequence[:histories])

    return np.array(prep_X_train)


def format_sequences_to_output(sequences):
    inputs = sequences.drop(columns=["opponent", "gene_0"]).values
    max_length = len(inputs[0])

    prep_y_train = []
    for histories in range(1, max_length + 1):
        for sequence in inputs:
            assert len(sequence) == max_length
            prep_y_train.append(sequence[histories - 1])

    return np.array(prep_y_train)

In [20]:
number_of_epochs = 1000
num_hidden_cells = 100
drop_out_rate = 0.2
num_of_steps = 205

In [48]:
outputs = pd.read_csv("data/targets.csv", index_col=0)
y = format_sequences_to_output(outputs)
sequences = pd.read_csv("data/sequences.csv", index_col=0)
inputs = format_sequences_to_input(sequences)
input_train, input_test, output_train, output_test = train_test_split(
    inputs, y, test_size=0.2, random_state=0
)

In [76]:
trainGen = batch_generator(input_train, output_train, num_of_steps)
testGen = batch_generator(input_test, output_test, num_of_steps)

In [111]:
for i in range(30):
    x, y = testGen.__next__()


In [116]:
x, y = testGen.__next__()

In [117]:
x.shape

(1085, 1, 1)

In [79]:
y.shape

(1085, 1)

In [ ]:




model = Sequential()

model.add(
    CuDNNLSTM(num_hidden_cells, return_sequences=True, input_shape=(None, 1))
)

model.add(CuDNNLSTM(num_hidden_cells))
model.add(Dropout(rate=drop_out_rate))

model.add((Dense(1, activation="sigmoid")))

if os.path.exists(file_name):
    model.load_weights(file_name)

model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)